## Load Disaster-related Statistics (BBS, 2015)

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, QuantileTransformer
import fhv
from tabula import read_pdf

In [143]:
unit_name = 'District'
sheet_name = 'Table_05'

df = pd.read_excel('./data/Disaster-related Statistics 2015.xlsx', 
                   sheet_name=sheet_name, 
                   skiprows=1,
                   header=[0,1])
sub1 = df[df.columns[:2]]
sub2 = df[df.columns[2:]]
sub1.columns = pd.MultiIndex.from_tuples([(c[0], '') for c in df[df.columns[:2]] ])
df = pd.concat([sub1,sub2], axis=1).set_index('Division/District')
assert df.isna().sum().sum() == 0
df

KeyError: "None of ['Division/District'] are in the columns"

[('Division/District', ''), ('Total Household', '')]

In [101]:
sub2

Household Income                                              \
        Agriculture Industry Service Business Day Labour Others   
0           1568629    15049  615785   644428    1422285  95084   
1            237377     2221  130578   173385     263255  11320   
2             43149      355   24499    22651      49034   3522   
3             28831      177   16564    37681      36886   1197   
4             58528      921   20344    37731      51953   2527   
..              ...      ...     ...      ...        ...    ...   
67           137657     1201   42679    48667     165517  14054   
68             5632       62    1611     2142       3792    232   
69            14412      240    8149     5211      21714   1810   
70            83490      561   24157    31693     114694   7465   
71            34124      338    8762     9622      25318   4547   

   Remittance Receiving               
               Received Not Received  
0                267928      4093333  
1                 25661       792475  
2                  4762       138448  
3                  4602       116734  
4                  5429       166575  
..                  ...          ...  
67                42635       367141  
68                 1214        12258  
69                 7977        43557  
70                19800       242259  
71                13644        69067  

[72 rows x 8 columns]

In [1]:
# ADD RERIGION

# Disaster-related Statistics (BBS, 2015) [7-10,23-28,30-56,61-66,69-74,86,91-125,130-139,143-146]
# Table 4: Distribution of household by main source of income and received remittance by division and district, 2014.
# Table 5: Distribution of main source oflighting and cooking fuel by division and district, 2014.
# Table 18: Distribution of annual household income from agricultural products by division and district, 2014.
# Table 20: Distribution of annual household income from non-agricultural sector by division and district, 2014.
# Table 22: Distribution of annual household income from other source by division and district, 2014.
# Table 23: Distribution of Disaster affected times of household by division, 2009-'14.
# Table 24: Distribution of affected households by disaster categories by division, 2009-'14.
# Table 25: Distribution of affected household and disaster categories by division and district, 2009-'14.
# Table 26: Distribution of household number of non working days due to last natural disaster by disaster categories and division, 2009-'14.
# Table 27: Distribution of Affected Household got early warning by disaster categories and division, 2009-'14.
# Table 28: Distribution of household got early warning by type of media, disaster categories and division, 2009-'14.
# Table 29: Distribution of affected area and loss of major crops by type of disaster categories and division, 2009-'14    
# Table 30: Distribution of affected area and value of loss and damage of minor crops by type of disaster categories and division, 2009-'14.
# Table 31: Distribution of affected area and loss of major crops by division and district, 2009-'14.
# Table 32: Distribution of affected area and loss of minor crops by division and district, 2009-'14.
# Table 35: Distribution of area and damage value of land by disaster categories and division, 2009-'14.
# Table 36: Distribution of area and damage value of land by division and district, 2009-'14.
# Table 39: Distribution of population suffering from sickness and injury by sex, disaster categories and division, 2009-'14.
# Table 40: Distribution of population suffering from sickness and injury by sex, age group and division, 2009-'14.
# Table 41: Distribution of population suffering from sickness and injury by sex, division and district, 2009-'14.  
# Table 42: Distribution of number of total children and sick children by division and district, 2009-'14.
# Table 48: Distribution of Children did not attend to School Due to Natural Disaster by Division and District, 2009-'14.
# Table 51: Distribution of disaster preparedness of household by disaster category and division, 2009-'14.
# Table 52: Distribution of disaster preparedness of household by division and district, 2009-'14.
# Table 53: Distribution of households having disaster precaution measures according to prior-disaster experience by disaster and division, 2009-'14.
# Table 54: Distribution of household preparedness during disaster period untill normal situation by disaster and division, 2009-'14.
# Table 55: Distribution of household preparedness during disaster period untill normal situation by division and district, 2009-'14.
# Table 56: Distribution of household taken action (precaution) during disaster period until normal situation by disaster and division, 2009-'14.
# Table 57: Distribution of population suffering from disease due to disaster by division and district, 2014.
# Table 58: Distribution of population suffering from disease due to natural disaster by sex, age group, division and district, 2014.
# Table 59: Distribution of Population Suffering from Disease Due to natural disaster by Type of Disease, Division and District, 2014.    
# Table 60: Distribution of household members suffering from disease before disaster by division and district, 2009-'14.
# Table 61: Distribution of household members suffering from disease during disaster period by division and district, 2009-'14
# Table 62: Distribution of household members suffering from disease post disaster period by division and district, 2009-'14.
# Table 63: Distribution of main probable cause of suffering from disease due to disaster by division and district, 2014.    
# Table 64: Distribution of source of household drinking water during disaster period by division and district, 2009-'14.
# Table 65: Distribution of other use of water (cooking, sewerage, cleanliness etc.) before disaster period by division and district, 2009-'14.
# Table 66: Distribution of other use water (cooking, sewerage, cleanliness etc.) during disaster period by division and district, 2009-'14.
# Table 67: Distribution of disease status due to insufficient drinking and other use of water supply during/after disaster period by division and district, 2009-'14.    
# Table 68: Distribution of cause of main disease due to insufficient drinking and other use of water supply during/after disaster period by division and district, 2009-'14.
# Table 71: Distribution of respondent's knowledge and perception about main impact of climate change by division and district, 2014.
# Table 73: Distribution of Respondent's knowledge and perception about disaster management by division and district, 2014.
# Table 74: Distribution of household received finantial/rehabiltation support from government/non-government agency during/post disaster period by division and district, 2009-'14
# Table 75: Distribution of household received financial/rehabilitation support from different organization/ office during/post disaster period by division and district, 2009-'14.
# Table 76: Distribution of households received loan from post disaster period by division and district, 2009-'14.
# Table A1: Standard error calculate of total income and total damage and loss by divisiond/ istrict. 
    
    

    

SyntaxError: invalid syntax (<ipython-input-1-3427511d581d>, line 4)

In [ ]:
### Type of House and Tenancy

In [ ]:
df = pd.read_excel('./data/union/Type of House and Tenancy.xls',
                   skiprows=11,header=0,index_col=0,skipfooter=8)

